In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


##Configuração do Ambiente no Colab

Artigo inicial  
https://harshityadav95.medium.com/how-to-run-cuda-c-or-c-on-google-colab-or-azure-notebook-ea75a23a5962

Link da configuração:  
https://gist.github.com/harshityadav95/e56c525dcc14eec0d4f64eac67ad5102  

Antes de cada bloco de execução é necessário colocar **%%cu**!


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jjd7jf43
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jjd7jf43
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=1ccb90fc803c0c2407cfd60f46f4667f560659c91113aca41c73565f7e559937
  Stored in directory: /tmp/pip-ephem-wheel-cache-31o0mmyy/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


## 1º Programa Cuda  
O programa apenas imprime o Id de cada thread.


In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>

__global__ void meu_kernel( void )
{
	printf( "Meu ID: %d\n" , threadIdx.x );
}

int main( )
{
	// Define a variável de captura de erros
	cudaError_t cudaStatus;

	// Informa o device a ser usado caso exista mais de 1
	cudaStatus = cudaSetDevice( 0 );

	// Testa a função cudaSetDevice retornou erro
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaSetDevice falhou!  Existe dispositivo com suporte a CUDA instalado?" );
    fprintf( stderr , "\n\n%s", cudaGetErrorString( cudaStatus ) );
		goto Error;
	}

	fprintf( stdout , "Inicio\n" );

	meu_kernel <<< 2 , 5 >>> ( );

	// Captura o último erro ocorrido
	cudaStatus = cudaGetLastError( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "meu_kernel falhou: %s\n" , cudaGetErrorString( cudaStatus ) );
		goto Error;
	}

	// Sincroniza a execução do kernel com a CPU
	cudaStatus = cudaDeviceSynchronize( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaDeviceSynchronize retornou erro %d após lançamento do kernel!\n" , cudaStatus );
		goto Error;
	}
	fprintf( stdout , "Fim\n" );
Error:
	// Executa a limpeza GPU
	cudaStatus = cudaDeviceReset( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaDeviceReset falhou!" );
		return 1;
	}

	return 0;
}

Inicio
Meu ID: 0
Meu ID: 1
Meu ID: 2
Meu ID: 3
Meu ID: 4
Meu ID: 0
Meu ID: 1
Meu ID: 2
Meu ID: 3
Meu ID: 4
Fim



## Detectar e inspecionar dispositivos compatíves com Cuda  
É possível inspecionar o sistema e obter informações sobre o hardware instalado.  
Isso auxiliará na criação e configuração do kernel.

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
int main( int argc , char** argv )
{

	fprintf( stdout , " CUDA Device Query\n" );

	int deviceCount = 0;
 
  // Testa se existem dispositivos compatíveis com Cuda
	cudaError_t cudaStatus = cudaGetDeviceCount( &deviceCount );

	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaGetDeviceCount retornou código: %d\n -> %s\n" , cudaStatus , cudaGetErrorString( cudaStatus ) );
		exit( 1 );
	}

	// A função retorna 0 caso não exista hardware que suporte cuda.
	if ( deviceCount == 0 )
	{
		fprintf( stdout , "Não há dispositivo compatível com CUDA\n" );
	}
	else
	{
		fprintf( stdout , "Detectado %d dispositivo(s) CUDA\n" , deviceCount );
	}
  return 0;
}

 CUDA Device Query
Detectado 1 dispositivo(s) CUDA



In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
int main( int argc , char** argv )
{

	fprintf( stdout , "CUDA Device Query\n" );

	int deviceCount = 0;
 
  // Testa se existem dispositivos compatíveis com Cuda
	cudaError_t cudaStatus = cudaGetDeviceCount( &deviceCount );

	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaGetDeviceCount retornou código: %d\n -> %s\n" , cudaStatus , cudaGetErrorString( cudaStatus ) );
		exit( 1 );
	}

	// A função retorna 0 caso não exista hardware que suporte cuda.
	if ( deviceCount == 0 )
	{
		fprintf( stdout , "Não há dispositivo compatível com CUDA\n" );
	}
	else
	{
		fprintf( stdout , "Detectado %d dispositivo(s) CUDA\n" , deviceCount );
	}
 
	int dev , driverVersion = 0 , runtimeVersion = 0;

	for ( dev = 0; dev < deviceCount; ++dev )
	{
		cudaSetDevice( dev );
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties( &deviceProp , dev );

		fprintf( stdout, "\nDevice %d: \"%s\"\n" , dev , deviceProp.name );
  
 		cudaDriverGetVersion( &driverVersion );
		cudaRuntimeGetVersion( &runtimeVersion );
		cudaDriverGetVersion( &driverVersion );
		cudaRuntimeGetVersion( &runtimeVersion );
		fprintf( stdout, "CUDA Driver Version / Runtime Version %d.%d / %d.%d\n" , driverVersion / 1000 , ( driverVersion % 100 ) / 10 , runtimeVersion / 1000 , ( runtimeVersion % 100 ) / 10 );
		fprintf( stdout, "CUDA Capability Major/Minor version number: %d.%d\n" , deviceProp.major , deviceProp.minor );
    fprintf( stdout, "QTD Multiprocessors: %d \n" , deviceProp.multiProcessorCount );
    fprintf( stdout, "Total constant memory:%zu bytes\n", deviceProp.totalConstMem );
    fprintf( stdout, "Total shared memory per block:%zu bytes\n", deviceProp.sharedMemPerBlock );
    fprintf( stdout, "Shared memory per multiprocessor:%zu bytes\n", deviceProp.sharedMemPerMultiprocessor );
    fprintf( stdout, "Number of registers available per block:%d\n", deviceProp.regsPerBlock );
  }

  return 0;
}

CUDA Device Query
Detectado 1 dispositivo(s) CUDA

Device 0: "Tesla T4"
CUDA Driver Version / Runtime Version 11.2 / 11.0
CUDA Capability Major/Minor version number: 7.5
QTD Multiprocessors: 40 
Total constant memory:65536 bytes
Total shared memory per block:49152 bytes
Shared memory per multiprocessor:65536 bytes
Number of registers available per block:65536



Outras informações contidas na estrutura cudaDeviceProp:

- deviceProp.warpSize
- deviceProp.maxThreadsPerMultiProcessor
- deviceProp.maxThreadsPerBlock
- deviceProp.maxThreadsDim
- deviceProp.maxGridSize
- deviceProp.memPitch
- deviceProp.textureAlignment

Link da documentação:
https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html

## Exemplo soma de 2 vetores



In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>

__global__ void addKernel( int* c , const int* a , const int* b )
{
	int i = threadIdx.x;
	c[ i ] = a[ i ] + b[ i ];
}

int main( )
{
	const int arraySize = 5;
	const int a[ arraySize ] = { 1, 2, 3, 4, 5 };
	const int b[ arraySize ] = { 10, 20, 30, 40, 50 };
	int c[ arraySize ] = { 0 };

	int* dev_a = 0;
	int* dev_b = 0;
	int* dev_c = 0;
	cudaError_t cudaStatus;

	// Alocar espaço na memória do device
	cudaStatus = cudaMalloc( ( void** ) &dev_c , arraySize * sizeof( int ) );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMalloc failed!" );
		goto Error;
	}

	cudaStatus = cudaMalloc( ( void** ) &dev_a , arraySize * sizeof( int ) );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMalloc failed!" );
		goto Error;
	}

	cudaStatus = cudaMalloc( ( void** ) &dev_b , arraySize * sizeof( int ) );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMalloc failed!" );
		goto Error;
	}

	// Copia os vetores do host para a device
	cudaStatus = cudaMemcpy( dev_a , a , arraySize * sizeof( int ) , cudaMemcpyHostToDevice );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMemcpy failed!" );
		goto Error;
	}

	cudaStatus = cudaMemcpy( dev_b , b , arraySize * sizeof( int ) , cudaMemcpyHostToDevice );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMemcpy failed!" );
		goto Error;
	}

	// Executar o kernel
	addKernel <<<1 , arraySize >> > ( dev_c , dev_a , dev_b );

	// Verificar se o kernel foi executado corretamente
	cudaStatus = cudaGetLastError( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "addKernel launch failed: %s\n" , cudaGetErrorString( cudaStatus ) );
		goto Error;
	}

	// Espera o kernel terminar e retorna quaisquer erros encontrados durante a execução
	cudaStatus = cudaDeviceSynchronize( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaDeviceSynchronize returned error code %d after launching addKernel!\n" , cudaStatus );
		goto Error;
	}

	// Copia o resultado do device para a memória do host.
	cudaStatus = cudaMemcpy( c , dev_c , arraySize * sizeof( int ) , cudaMemcpyDeviceToHost );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaMemcpy failed!" );
		goto Error;
	}

	printf( "{1,2,3,4,5} + {10,20,30,40,50} = {%d,%d,%d,%d,%d}\n" , c[ 0 ] , c[ 1 ] , c[ 2 ] , c[ 3 ] , c[ 4 ] );

	// Limpa a memória
Error:
	cudaFree( dev_c );
	cudaFree( dev_a );
	cudaFree( dev_b );

	cudaStatus = cudaDeviceReset( );
	if ( cudaStatus != cudaSuccess )
	{
		fprintf( stderr , "cudaDeviceReset failed!" );
		return 1;
	}

	return 0;
}



{1,2,3,4,5} + {10,20,30,40,50} = {11,22,33,44,55}

